# Create DataFrame

In [1]:
import pandas as pd
import requests
import difflib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from matplotlib import pyplot


# Define your API key and base URL
api_key = "d0a63712c67a451ca1cafef886413857"
current_week_url = "https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek"
schedule_url = 'https://api.sportsdata.io/v3/nfl/scores/json/SchedulesBasic'
get_stadium_url = 'https://api.sportsdata.io/v3/nfl/scores/json/Schedules'
base_url = "https://api.sportsdata.io/v3/nfl/stats/json/PlayerGameStatsByWeek"

# Create stats given week of interest, weeks_back

In [2]:
def get_current_week():
  response = requests.get(f'{current_week_url}?key={api_key}')
  if response.status_code == 200:
    return int(response.json())

In [3]:
def get_stats(weeks_back, start_week=get_current_week()):
  data_list = []
  for week in range(start_week, start_week - weeks_back, -1):
    response = requests.get(f'{base_url}/2023/{week}?key={api_key}')

    # Check if the response is successful
    if response.status_code == 200:
      # Append data to the list
      data_list.extend(response.json())
    else:
      print(f'Failed to get week {week}')
  df = pd.DataFrame(data_list)
  return df

In [7]:
last_week_interested = 13
weeks_back = 5
week_to_predict = 14

In [8]:
p5_full = get_stats(weeks_back, last_week_interested)

In [9]:
p5 = p5_full.copy()

In [10]:
irrelevant_features = [
    # Defensive and Special Teams Stats
    'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns', 'RushingLong',
    'ReceivingTargets', 'Receptions', 'ReceivingYards', 'ReceivingYardsPerReception', 'ReceivingTouchdowns', 'ReceivingLong',
    'SoloTackles', 'AssistedTackles', 'TacklesForLoss', 'Sacks', 'SackYards', 'QuarterbackHits', 'PassesDefended',
    'FumblesForced', 'FumblesRecovered', 'FumbleReturnYards', 'FumbleReturnTouchdowns', 'Interceptions',
    'InterceptionReturnYards', 'InterceptionReturnTouchdowns', 'BlockedKicks', 'SpecialTeamsSoloTackles',
    'SpecialTeamsAssistedTackles', 'MiscSoloTackles', 'MiscAssistedTackles', 'PuntReturns', 'PuntReturnYards',
    'PuntReturnYardsPerAttempt', 'PuntReturnTouchdowns', 'PuntReturnLong', 'KickReturns', 'KickReturnYards',
    'KickReturnYardsPerAttempt', 'KickReturnTouchdowns', 'KickReturnLong', 'PuntReturns', 'PuntReturnYards',
    'PuntReturnYardsPerAttempt', 'PuntReturnTouchdowns', 'PuntReturnLong', 'BlockedKickReturnYards',
    'FieldGoalReturnYards', 'KickReturnFairCatches', 'PuntReturnFairCatches', 'PuntTouchbacks', 'PuntInside20',
    'PuntNetAverage', 'KickReturnFairCatches', 'PuntReturnFairCatches', 'PuntTouchbacks', 'PuntInside20',
    'PuntNetAverage', 'BlockedKickReturnYards', 'FieldGoalReturnYards', 'PuntNetYards', 'SpecialTeamsFumblesForced',
    'SpecialTeamsFumblesRecovered', 'MiscFumblesForced', 'MiscFumblesRecovered','BlockedKicks',
    'SpecialTeamsSoloTackles',
    'SpecialTeamsAssistedTackles',
    'MiscSoloTackles',
    'MiscAssistedTackles',
    'Punts',
    'PuntYards',
    'PuntAverage',
    'FieldGoalsAttempted',
    'FieldGoalsMade',
    'FieldGoalsLongestMade',
    'ExtraPointsMade',
    'BlockedKickReturnTouchdowns', 'DefensiveSnapsPlayed', 'DefensiveTeamSnaps', 'DefensiveTouchdowns', 'ExtraPointsAttempted',
    'ExtraPointsHadBlocked', 'FieldGoalPercentage', 'FieldGoalReturnTouchdowns', 'FieldGoalsHadBlocked', 'FieldGoalsMade0to19',
    'FieldGoalsMade20to29', 'FieldGoalsMade30to39', 'FieldGoalsMade40to49', 'FieldGoalsMade50Plus', 'Fumbles', 'FumblesLost',
    'FumblesOutOfBounds', 'FumblesOwnRecoveries', 'PuntLong', 'PuntsHadBlocked', 'ReceivingYardsPerTarget', 'ReceptionPercentage',
    'Safeties', 'SafetiesAllowed', 'Season', 'SeasonType', 'SpecialTeamsSnapsPlayed', 'SpecialTeamsTeamSnaps', 'SpecialTeamsTouchdowns',
    'Started', 'Tackles', 'TwoPointConversionPasses', 'TwoPointConversionReceptions', 'TwoPointConversionReturns', 'TwoPointConversionRuns',
    'Week',

    # Fantasy Points, Salaries, and Irrelevant Identifiers
    'FantasyPoints', 'FantasyPointsPPR', 'FanDuelSalary', 'DraftKingsSalary', 'FantasyDataSalary', 'VictivSalary',
    'YahooSalary', 'FantasyPointsFanDuel', 'FantasyPointsDraftKings', 'FantasyPointsYahoo', 'FantasyPointsFantasyDraft',
    'FanDuelPosition', 'DraftKingsPosition', 'YahooPosition', 'FanDuelSalary', 'DraftKingsSalary', 'FantasyDraftSalary',
    'FantasyDraftPosition', 'TeamID', 'OpponentID', 'Day', 'DateTime', 'Updated', 'InjuryStartDate', 'InjuryNotes',
    'InjuryPractice', 'InjuryPracticeDescription', 'DeclaredInactive', 'PlayerID', 'Number',

    # Opponent-Related Identifiers
    'GlobalOpponentID', 'ScoreID', 'GlobalTeamID', 'GlobalOpponentID',

    # Additional Stats
    'BlockedKickReturnTouchdowns', 'ExtraPointsAttempted', 'ExtraPointsHadBlocked', 'ExtraPointsMade', 'FieldGoalPercentage',
    'FieldGoalReturnTouchdowns', 'FieldGoalsAttempted', 'FieldGoalsHadBlocked', 'FieldGoalsLongestMade', 'FieldGoalsMade',
    'FieldGoalsMade0to19', 'FieldGoalsMade20to29', 'FieldGoalsMade30to39', 'FieldGoalsMade40to49', 'FieldGoalsMade50Plus',
    'PuntAverage', 'PuntLong', 'PuntYards', 'Punts', 'PuntsHadBlocked', 'SpecialTeamsSnapsPlayed', 'SpecialTeamsTeamSnaps',
    'SpecialTeamsTouchdowns', 'DefensiveSnapsPlayed', 'DefensiveTeamSnaps', 'DefensiveTouchdowns', 'Fumbles', 'FumblesLost',
    'FumblesOutOfBounds', 'FumblesOwnRecoveries', 'PassingInterceptions', 'PassingSackYards', 'PassingSacks', 'Safeties',
    'SafetiesAllowed', 'Tackles',

    # Game-Related Identifiers
    'GameDate', 'GameKey', 'GlobalGameID', 'ShortName', 'FantasyPosition', 'IsGameOver', 'ScoringDetails', 'PlayerGameID',
    'OpponentRank', 'Played', 'Team', 'Opponent', 'Position', 'PositionType'
]


In [11]:
features = p5.columns.difference(irrelevant_features)

# Preprocess


In [12]:
LINE = 29.5

In [13]:
def preprocess(line):
  global p5
  p5 = p5[p5['Position'] == 'QB']
  p5 = p5[p5['Started'] == 1]
  p5[f'OverPassingAttempts'] = (p5['PassingAttempts'] > line).astype(int)


  p5 = p5[list(features) + ['OverPassingAttempts']]

  # get dummies
  #label_encoder = LabelEncoder()

  dummies = []
  appender = 'For'
  for col in p5.columns:
      if p5[col].dtype == 'object':
        try:
          dummy = pd.get_dummies(p5[col])
          if 'ARI' in dummy.columns:
            new_names = {old: f'{old}{appender}' for old in dummy.columns}
            appender = 'Against'
            dummy = dummy.rename(columns=new_names)
          dummies.append(dummy)
          p5 = p5.drop(col, axis=1)
        except:
          p5 = p5.drop(col, axis=1)
  p5 = pd.concat([p5] + dummies, axis=1)
  p5 = p5.apply(pd.to_numeric)
  p5 = p5.drop([True, 'Scrambled', 'Scrambled'], axis=1)
preprocess(LINE)

# Train Model given line

In [14]:
# train
X = p5.drop(['OverPassingAttempts'],axis=1)
y = p5['OverPassingAttempts']

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=4000,solver='lbfgs')

model.fit(X_train, y_train)


LogisticRegression(max_iter=4000)

In [15]:
p = model.predict(X_test)
print(classification_report(y_test, p))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.92      0.86      0.89        14

    accuracy                           0.90        29
   macro avg       0.90      0.90      0.90        29
weighted avg       0.90      0.90      0.90        29



In [16]:
importance = model.coef_.flatten()

In [17]:
pyplot.rcParams['figure.figsize'] = (100,100)
pyplot.barh(p5.drop('OverPassingAttempts',axis=1).columns, importance, color='g')
pyplot.show()


# Get next game data

In [18]:
# get TeamId from playerID
def get_teamid(player_id):
  try:
    return p5[p5['PlayerID'] == player_id].iloc[0]['TeamID']
  except:
    return 2


# get OpponentID from playerID and week
def get_oppid(player_id, week):
  teamid = get_teamid(player_id)
  response = requests.get(f'{schedule_url}/2023?key={api_key}')
  if response.status_code == 200:
    schedules = response.json()

    # Filter the schedules for the specified teamID and week
    for game in schedules:
      if game['Week'] == week and (game['AwayTeamID'] == teamid or game['HomeTeamID'] == teamid):
        opponent = game['AwayTeamID'] if game['HomeTeamID'] == teamid else game['HomeTeamID']
        return opponent

    return f"No game found for Team ID {teamid} in Week {week}"
  else:
    return f"Failed to fetch data. Status code: {response.status_code}"

# if home for player and week
def get_ifhome(player_id, week):
  teamid = get_teamid(player_id)
  response = requests.get(f'{schedule_url}/2023?key={api_key}')
  if response.status_code == 200:
    schedules = response.json()

    # Filter the schedules for the specified teamID and week
    for game in schedules:
      if game['Week'] == week and (game['AwayTeamID'] == teamid or game['HomeTeamID'] == teamid):
        return 1 if game['HomeTeamID'] == teamid else 0

    return f"No game found for Team ID {teamid} in Week {week}"
  else:
    return f"Failed to fetch data. Status code: {response.status_code}"

# Prediction

In [19]:
def get_mean(name, col):
  return X[X[name] == 1][col].mean()


In [21]:
# stuff i know
knowns = ['Activated', 'OpponentPositionRank', 'Season', 'Week', 'AWAY', 'HOME', 'Artificial', 'Grass', 'Stadium']

# and TEAMHome and AWAYHome

In [22]:
stadiums_list = [
    'AT&T Stadium',
    'Acrisure Stadium',
    'Allegiant Stadium',
    'Arrowhead Stadium',
    'Bank of America Stadium',
    'Caesars Superdome',
    'Cleveland Browns Stadium',
    'Deutsche Bank Park',
    'Empower Field at Mile High',
    'EverBank Stadium',
    'FedEx Field',
    'Ford Field',
    'Gillette Stadium',
    'Hard Rock Stadium',
    'Highmark Stadium',
    'Lambeau Field',
    "Levi's Stadium",
    'Lincoln Financial Field',
    'Lucas Oil Stadium',
    'Lumen Field',
    'M&T Bank Stadium',
    'Mercedes-Benz Stadium',
    'MetLife Stadium',
    'NRG Stadium',
    'Nissan Stadium',
    'Paycor Stadium',
    'Raymond James Stadium',
    'SoFi Stadium',
    'Soldier Field',
    'State Farm Stadium',
    'U.S. Bank Stadium'
]

In [23]:
def predict(name, team, opp_team, stadium, week):
  global model, X
  data = {}
  data['Activated'] = 1
  data['OpponentPositionRank'] = 0
  data['OpponentRank'] = 2
  data['Season'] = 2023
  data['Week'] = week
  data['AWAY'] = 0
  data['HOME'] = 1
  data['Artificial'] = 1
  if data['Artificial'] == 1:
    data['Grass'] = 0
  else:
    data['Grass'] = 1

  for col in X.columns.difference(knowns):
    if 'Home' in col or 'Away' in col or col in stadiums_list:
      data[col] = 0
    else:
      data[col] = get_mean(name, col)

  data[stadium] = 1

  if data['HOME'] == 1:
    data[f'{team}Home'] = 1
    data[f'{opp_team}Away'] = 1
  else:
    data[f'{team}Away'] = 1
    data[f'{opp_team}Home'] = 1

  data_ordered = [data[col] for col in X.columns]
  p = model.predict(pd.DataFrame([data_ordered], columns=X.columns, index=[0]))
  return 'Over' if p == 1 else 'Under'

In [25]:
predict('Brock Purdy', 'SF', 'SEA', "Levi's Stadium", 14)

'Under'